In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm

import myfunctions
import myplot
import mylearn
import matplotlib.pyplot as plt

logitech_step1_fast = ('logitech-fast/niels-step_1-1617765899871', 0)
logitech_step1_slow = ('logitech-slow/niels-step_1-1617765658270', 60, True)
lenovo_slow = ('lenovo-slow-easy/niels-step_5-1617952576650', 0)
lenovo_slow2 = ('lenovo-slow-easy/niels-step_5-1618197371437', 0)
lenovo_hard = ('lenovo-hard/niels-step_1-1618199054005', 0)
noweb1 = ('winform/winform-1618904428438', 60, False)
noweb2 = ('winform/winform-1619147742756', 75, False)

In [ ]:
path, sync_adjustment, adjust_missing_samples = noweb2
d = myfunctions.getData(path, sync_adjustment=sync_adjustment, adjust_missing_samples=adjust_missing_samples,
                       keystroke_min_peak_level=0.1)

In [ ]:
# myplot.plotKeystrokeContext(d, 81, 75)

# for i in range(21, 31):
#     myplot.plotKeystroke(d, i)

# myplot.plotLostSamples(d)

# d.keys()

set([x[3] for x in d['down_events']])

In [ ]:
mylearn.testClustering(d,
    keep=None,
    features=['mfcc_mean', 'mfcc_std'])


In [ ]:
# This worked pretty well
mylearn.testClustering(myfunctions.getData('winform/winform-1619147742756',
                                           sync_adjustment=75,
                                           adjust_missing_samples=False,
                                           keystroke_min_peak_level=0.1),
    keep=None,
    features=['mfcc_mean', 'mfcc_std'])


In [ ]:
def findNumberOfClusters(features):
    x = list(range(2, 20))
    scores = []
    for n_clusters in tqdm(x):
        kmeans = sklearn.cluster.KMeans(n_clusters)
        kmeans.fit(features)
        kmeans_score = kmeans.score(features)
        clustering = kmeans.predict(features)
        kmeans_silhouette_score = silhouette_score(features, clustering)

        gm = sklearn.mixture.GaussianMixture(n_clusters)
        gm.fit(features)
        gm_score = gm.score(features)
        gm_bic_score = gm.bic(features)
        
        scores.append((kmeans_score, kmeans_silhouette_score, gm_score, gm_bic_score))
    return x, scores

features = mylearn.getConcatenatedFeatures(d, ['mfcc_max', 'mfcc_mean', 'mfcc_std'])
number_of_clusters,scores = findNumberOfClusters(mylearn.getConcatenatedFeatures(d, ['mfcc_max', 'mfcc_mean', 'mfcc_std']))

for (i, s) in enumerate(['kmeans_score', 'kmeans_silhouette_score', 'gm_score', 'gm_bic_score']):
    plt.figure()
    plt.title(s)
    plt.scatter(number_of_clusters, [x[i] for x in scores])
    plt.show()

In [ ]:
wav = logitech_step1_slow['keystroke_wavs'][0]
print( librosa.feature.spectral_centroid(wav, 48000, win_length=1000, hop_length=2).shape )
print( librosa.feature.chroma_stft(wav, 48000, win_length=1000, hop_length=2).shape )
print( librosa.feature.zero_crossing_rate(wav, frame_length=1000, hop_length=2).shape )
print( list(librosa.feature.zero_crossing_rate(wav, frame_length=2, hop_length=2)[0]) )

In [ ]:

for i in range(10, 17):
    myplot.plotKeystroke(d, i)

In [ ]:
def p(key):
    indexes = [i for i, k in enumerate(d['keystrokes']) if k[0]==key]
    for i in indexes:
        myplot.plotKeystroke(d, i)
p('Return')
p('Oemplus')